In [1]:
from transformers import Qwen2VLForConditionalGeneration, AutoTokenizer, AutoProcessor
from qwen_vl_utils import process_vision_info
import torch

import os
from tqdm import tqdm
import json
import signal
import argparse
import wandb
import pandas as pd

import copy
import re


import sys
from collections import defaultdict
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score,confusion_matrix
from pathlib import Path

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/conda/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/opt/conda/lib/python3.10/site-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
/opt/conda/lib/python3.10/site-packages/torchvision/datapoints/__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to 

In [2]:
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [11]:
sys.argv = [
    'notebook',  
    '--modelname' , 'Qwen/Qwen2-VL-72B-Instruct' , 
    '--data', '/root/home/data',
    '--data_path','/mnt/Gbenga_Enemy/ramy/results/test_Qwen2-VL-72B-Instruct-QAs.json',
    '--data_path_files','/mnt/Gbenga_Enemy/ramy/WACV-2025-Workshop-ViGIR',
    '--results_dir', '/mnt/Gbenga_Enemy/ramy/results',
]

In [12]:
parser = argparse.ArgumentParser(description="A script to run V-LLMs on different image classification datasets")

In [13]:
parser.add_argument("--modelname", type=str, required=True, help="The name of the V-LLM model")
parser.add_argument("--data", type=str, required=True, help="Path to the data")
parser.add_argument("--data_path", type=str, required=True, help="Path to the image data dir")
parser.add_argument("--data_path_files", type=str, required=True, help="Path to the image data dir")
parser.add_argument("--results_dir", type=str, required=True, help="Folder name to save results")

args = parser.parse_args()

In [14]:
def prepare_inference_inputs_textOnly(prompt):
    messages = [
        {
            "role": "user",
            "content": [
                {"type": "text", "text": prompt},
            ],
        }
    ]
    

    text = processor.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    image_inputs, video_inputs = process_vision_info(messages)
    inputs = processor(  text=[text] ,      padding=True,    return_tensors="pt",)
    
    return inputs

In [7]:
processor = AutoProcessor.from_pretrained("Qwen/Qwen2-VL-72B-Instruct")

model = Qwen2VLForConditionalGeneration.from_pretrained(
    args.modelname,
    torch_dtype=torch.bfloat16,
    attn_implementation="flash_attention_2",
    device_map="auto",
).eval()

`Qwen2VLRotaryEmbedding` can now be fully parameterized by passing the model config through the `config` argument. All other arguments will be removed in v4.46
Loading checkpoint shards: 100%|████████████████████████████████████████████████████████| 38/38 [09:08<00:00, 14.45s/it]


In [15]:
# Load test set:
file_path = os.path.join(args.data_path)
with open(file_path, 'r') as file:
    data = json.load(file)

print('Number of Annotated GT Images: ', len(data.keys()))
data_keys_test = list(data.keys())
print('Number of Annotated GT Images (List): ', len(data_keys_test))

Number of Annotated GT Images:  311
Number of Annotated GT Images (List):  311


In [16]:
file_path = os.path.join(args.data_path)
print(file_path)

/mnt/Gbenga_Enemy/ramy/results/test_Qwen2-VL-72B-Instruct-QAs.json


In [17]:
data

{'415': [[{'label': '4', 'labelers': ['Ali', 'Dr.Lory']},
   'Given these six images of the exact same area and collected over a period of 10 years, do you see a low point in the terrain? Answer with yes or no only!',
   ['Yes.']],
  [{'label': '4', 'labelers': ['Ali', 'Dr.Lory']},
   'Given these six images of the exact same area and collected over a period of 10 years, do narrow, winding paths or channels appear? Answer with yes or no only!',
   ['Yes.']],
  [{'label': '4', 'labelers': ['Ali', 'Dr.Lory']},
   'Given these six images of the exact same area and collected over a period of 10 years, are there winding paths that become intermittent recurrent? Answer with yes or no only!',
   ['Yes.']],
  [{'label': '4', 'labelers': ['Ali', 'Dr.Lory']},
   'Given these six images of the exact same area and collected over a period of 10 years, are there any linear depressions or ruts which appear more pronounced along natural drainage lines or slopes? Answer with yes or no only!',
   ['Yes.

In [18]:
data_reformated = {}

for key, item in data.items():
    q_and_a = []
    for i in range(len(item)):
        tmp = item[i][1:]   
        q_and_a.append(tmp)

    data_reformated[key] = q_and_a
    

In [19]:
data_prompt = {}
for key, item in data_reformated.items():
    prompt = "\n".join([f"Q: {qa[0]}\nA: {qa[1][0]}" for qa in item])
    data_prompt[key] = [prompt]

In [20]:
data_prompt

{'415': ['Q: Given these six images of the exact same area and collected over a period of 10 years, do you see a low point in the terrain? Answer with yes or no only!\nA: Yes.\nQ: Given these six images of the exact same area and collected over a period of 10 years, do narrow, winding paths or channels appear? Answer with yes or no only!\nA: Yes.\nQ: Given these six images of the exact same area and collected over a period of 10 years, are there winding paths that become intermittent recurrent? Answer with yes or no only!\nA: Yes.\nQ: Given these six images of the exact same area and collected over a period of 10 years, are there any linear depressions or ruts which appear more pronounced along natural drainage lines or slopes? Answer with yes or no only!\nA: Yes.\nQ: Given these six images of the exact same area and collected over a period of 10 years, are there narrow and shallow channels which appear intermittently deeper or more indented into the soil? Answer with yes or no only!\n

In [29]:
count = 0
saving_response=copy.deepcopy(data_prompt)

for key, info in tqdm(data_prompt.items()):


    question = "Based on the following questions and their answers, determine if there is evidence of an ephemeral gully in the observed area. Carefully analyze all the questions and the responses to assess.\n\n"
    question += info[0]
    question += "\n\nAfter considering these responses, provide a clear conclusion: Is there evidence of an ephemeral gully? Answer with yes or no only."
    
        
    count+=1

    inputs=prepare_inference_inputs_textOnly(question)
    inputs = inputs.to("cuda")

    generated_ids = model.generate(**inputs, max_new_tokens=128)
    generated_ids_trimmed = [out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)]
    response = processor.batch_decode(generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False)
    print(key,response)
    
    saving_response[key].append(response[0])

    
    
# with open(os.path.join(args.results_dir,f'test_{args.modelname[-21:]}-Aggregating-QAs-Dispose.json'), "w") as file:
#     json.dump(saving_response, file)

  0%|▎                                                                                  | 1/311 [00:05<28:05,  5.44s/it]

415 ['Yes.']


  1%|▌                                                                                  | 2/311 [00:06<14:03,  2.73s/it]

1020 ['No.']


  1%|▊                                                                                  | 3/311 [00:07<09:32,  1.86s/it]

...

212 ['Yes.']


100%|████████████████████████████████████████████████████████████████████████████████▋| 310/311 [04:20<00:00,  1.21it/s]

131 ['No.']


100%|█████████████████████████████████████████████████████████████████████████████████| 311/311 [04:21<00:00,  1.19it/s]

1133 ['Yes.']


In [21]:
# with open('/mnt/Gbenga_Enemy/ramy/results/test_Qwen2-VL-72B-Instruct-Aggregating-QAs.json', 'r') as file:
#     saving_response=json.load(file)

In [22]:
def extract_first_yes_no(text):
    # Use regular expressions to find "Yes" or "No" (case-insensitive)
    match = re.search(r'\b(Yes|No)\b', text, re.IGNORECASE)
    if match:
        # Return the first match in its original case
        return match.group(0)
    return None

In [30]:
saving_response_reformat = {}

for key, item in saving_response.items():
    text = extract_first_yes_no(item[-1])

    print(f"Image: {key} - Answer: {text}")
    
    if text == 'Yes':
        saving_response_reformat[key] = ["4"]

    else:
        saving_response_reformat[key] = ["0"]

Image: 415 - Answer: Yes
Image: 1020 - Answer: No
Image: 105 - Answer: No
Image: 439 - Answer: Yes
Image: 914 - Answer: Yes
Image: 1099 - Answer: Yes
Image: 1065 - Answer: Yes
Image: 373 - Answer: Yes
Image: 166 - Answer: Yes
Image: 396 - Answer: Yes
Image: 837 - Answer: Yes
Image: 685 - Answer: Yes
Image: 226 - Answer: Yes
Image: 956 - Answer: Yes
Image: 70 - Answer: Yes
Image: 604 - Answer: Yes
Image: 1067 - Answer: Yes
Image: 118 - Answer: Yes
Image: 774 - Answer: Yes
Image: 521 - Answer: Yes
Image: 910 - Answer: No
Image: 975 - Answer: Yes
Image: 352 - Answer: No
Image: 761 - Answer: No
Image: 1007 - Answer: Yes
Image: 428 - Answer: Yes
Image: 1038 - Answer: Yes
Image: 50 - Answer: Yes
Image: 838 - Answer: Yes
Image: 126 - Answer: Yes
Image: 1078 - Answer: Yes
Image: 944 - Answer: Yes
Image: 532 - Answer: Yes
Image: 1041 - Answer: No
Image: 540 - Answer: Yes
Image: 128 - Answer: Yes
Image: 722 - Answer: Yes
Image: 478 - Answer: Yes
Image: 639 - Answer: Yes
Image: 668 - Answer: Yes


In [24]:
valid_file=os.path.join(args.data_path_files,'Validation_Set.json')
test_file=os.path.join(args.data_path_files,'Testing_Set.json')

with open(valid_file, 'r') as file:
    valid = json.load(file) 

with open(test_file, 'r') as file:
    test = json.load(file) 

In [31]:
ground_truth=[]
model_response=[]

for key,info in test.items():
    if int(info['label']) ==4:
        ground_truth.append(1)
    else:
        ground_truth.append(0)
    
    if int(saving_response_reformat[key][0])==4:
        model_response.append(1)
    else:
        model_response.append(0)

In [32]:
#################### This results for Llama VQA + Qwen Aggregating #################33
y_true=ground_truth
y_pred=model_response

precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred, average='macro')
accuracy = accuracy_score(y_true, y_pred)
tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()

print("True Positives (TP):", tp)
print("False Positives (FP):", fp)
print("False Negatives (FN):", fn)
print("True Negatives (TN):", tn)


print("Precision:", precision)
print("Recall:", recall)


print("Accuracy:", accuracy)

f1_per_class = f1_score(y_true, y_pred, average=None)
print(f1_per_class)

print("F1 Score (Macro):", f1)



True Positives (TP): 173
False Positives (FP): 103
False Negatives (FN): 4
True Negatives (TN): 31
Precision: 0.6268115942028986
Recall: 0.9774011299435028
Accuracy: 0.6559485530546624
[0.36686391 0.76379691]
F1 Score (Macro): 0.5653304074088588


[0.36686391 0.76379691]


In [33]:
l= [
            0,
            0,
            1,
            0,
            0,
            1,
            1,
            0,
            0,
            0,
            0,
            1,
            0,
            0,
            0
        ]
